In [1]:
import observational_fear.load as load
from observational_fear.events import get_freeze_starts
from neurobox.wide_transforms import resample
from neurobox.compose import Pipeline
from observational_fear.nbox_transforms import (
    align_to_data_by,
    summarize_prepost,
    wilcoxon_by_cell
)
from observational_fear.stats import auc
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")



DATA_DIR = Path(r"F:\OFL\one-p experiment\data")
FIG_DIR = Path(r"C:\Users\roryl\repos\observational-fear\figs")

In [6]:
def align_to_freeze(
    data_dir, session, t_before, t_after, start_stop, role="obs", rotate=False, min_freeze_diff=11
):
    df_cells = load.load_cells(data_dir).rename(columns={"mouse": "mouse_name"})
    df_cells["new_id"] = df_cells["new_id"].astype("str")
    if start_stop == "start":
        freeze_getter = load.load_freeze_starts
        freeze_col = "freeze_start"
    elif start_stop == "stop":
        freeze_getter = load.load_freeze_stops
        freeze_col = "freeze_stop"
    events = freeze_getter(data_dir, session, role=role, min_diff=min_freeze_diff)
    traces = load.load_traces_long(data_dir, session, rotate=rotate).merge(df_cells)
    df_aligned = align_to_data_by(
        df_data=traces,
        df_events=events,
        df_data_time_col="time",
        df_data_cell_col="new_id",
        df_data_value_col="value",
        time_before_event=t_before,
        time_after_event=t_after,
        df_data_group_col="mouse_name",
        df_events_timestamp_col=freeze_col,
        df_events_group_colname="mouse_name",
        precision=1,
    )
    return df_aligned


def freeze_wilcoxon_prepost(
    data_dir, session, t_before, t_after, start_stop, role="obs", rotate=False, min_freeze_diff=11
):
    df_aligned = align_to_freeze(
        data_dir,
        session,
        t_before,
        t_after,
        start_stop=start_stop,
        role=role,
        rotate=rotate,
        min_freeze_diff=min_freeze_diff,
    )
    df_prepost = summarize_prepost(
        df_aligned,
        trial_col="event",
        summary_func=auc,
        cell_col="new_id",
        value_col="value",
    )
    df_res = wilcoxon_by_cell(df_prepost, cell_col="new_id")
    return df_res



In [7]:
df_res_start = freeze_wilcoxon_prepost(DATA_DIR, "day2", start_stop="start", t_before=5, t_after=5)
df_res_stop = freeze_wilcoxon_prepost(DATA_DIR, "day2", start_stop="stop", t_before=5, t_after=5)

In [11]:
df_res_start = freeze_wilcoxon_prepost(DATA_DIR, "day2", start_stop="start", t_before=5, t_after=5, rotate=True)


In [12]:
(df_res_start.p < 0.05).mean()

0.0